# Fase 1 - Business Understanding

El banco Monopoly, con una larga trayectoria en el mercado chileno, ha sido recientemente adquirido por la entidad financiera internacional Dormammu. Ante este cambio, Dormammu ha encomendado a su equipo de ingenieros de datos la tarea de analizar exhaustivamente la base de clientes de Monopoly. El objetivo principal es comprender los hábitos financieros de esta nueva clientela y diseñar estrategias personalizadas que permitan una integración exitosa al ecosistema de Dormammu. Para ello, se ha puesto a disposición de los ingenieros una base de datos que abarca un año de información transaccional de una muestra representativa de clientes de Monopoly. La labor del equipo consiste en limpiar y analizar estos datos, identificando patrones de consumo, preferencias y necesidades financieras. Con esta información, se busca generar insights valiosos que permitan a Dormammu diseñar una oferta de productos y servicios a medida, optimizando así la experiencia del cliente y maximizando la rentabilidad.

Para esto, como equipo analista de datos vamos a darle respuesta a las siguientes incognitas:

- ¿Cuáles son los 3 meses de mayor uso de tarjetas de crédito a nivel nacional e internacional?
- ¿Existen diferentes grupos de clientes basados en su comportamiento financiero, como el uso de tarjetas de crédito, nivel de deuda y productos contratados?
- ¿Cuál es la probabilidad de que los clientes con ciertas características demográficas o comportamiento histórico caigan en mora?
- ¿Los clientes con mayor antigüedad o que usan múltiples productos son más fieles al banco?
- ¿Los clientes con dualidad (dos o más tarjetas de crédito) son más propensos a utilizar servicios adicionales como avances en cuotas o compras internacionales?

Con lo expuesto anteriormente, se espera responder satisfactoriamente a estas preguntas para que el nuevo dueño del banco pueda tener información sólida y conocer a mayor profundidad las interacciones económicas de sus clientes.

# Fase 2 - Data Understanding

### A continuación se importarán las librerías necesarias para trabajar en esta metodología.




In [ ]:
#Importación de librerias

%pip install pyarrow
%pip install tpot
%pip install --upgrade setuptools
%pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
%pip install ipywidgets
%pip install imbalanced-learn

import os
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt

from tpot import TPOTClassifier, TPOTRegressor
from imblearn.over_sampling import SMOTE, SMOTENC
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy import stats 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, classification_report
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


# AUTOMATIZACIÓN *
### Se realiza conversión del archivo "Base_clientes_Monopoly.xlsx", al formato de archivo '.parquet' en caso de no existir en las rutas predefinidas, para una mayor eficiencia en la carga y análisis de los datos. MODIFICAR la ruta de salida para el archivo '.parquet' si este no existe en el sistema en el que se ejecuta este Notebook.

In [ ]:
# Definir las posibles rutas de salida para el archivo Parquet
ruta_parquet_1 = "/Users/herna/Desktop/Duoc_UC/6to SEMESTRE/Machine_Learning/ET/Base_clientes_Monopoly.parquet"
ruta_parquet_2 = "/Users/new11/Documents/DUOC - Ing. Informática/2024/2024-2/Machine Learning/Base_clientes_Monopoly.parquet"
ruta_parquet_3 = "/Users/abrahamrubilaralbarran/Desktop/DataSet Monopoly /Base_clientes_Monopoly.parquet"

# Verificar si el archivo Parquet ya existe en alguna de las rutas de salida
if os.path.exists(ruta_parquet_1):
    ruta_parquet = ruta_parquet_1
    print(f"Archivo .parquet encontrado en {ruta_parquet}. Cargando directamente.")
elif os.path.exists(ruta_parquet_2):
    ruta_parquet = ruta_parquet_2
    print(f"Archivo .parquet encontrado en {ruta_parquet}. Cargando directamente.")
elif os.path.exists(ruta_parquet_3):
    ruta_parquet = ruta_parquet_3
    print(f"Archivo .parquet encontrado en {ruta_parquet}. Cargando directamente.")
else:
    # Si no existe el archivo .parquet, procede con la carga y conversión desde el archivo Excel
    print("'Base_clientes_Monopoly.parquet' no encontrado. Procediendo a cargar y convertir desde el 'Base_clientes_Monopoly.xlsx'.")
    
    # Definir las rutas de entrada del archivo Excel
    ruta_excel_1 = "/Users/herna/Desktop/Duoc_UC/6to SEMESTRE/Machine_Learning/ET/Base_clientes_Monopoly.xlsx"
    ruta_excel_2 = "/Users/new11/Documents/DUOC - Ing. Informática/2024/2024-2/Machine Learning/Dataset Monopoly.xlsx"
    ruta_excel_3 = "/Users/abrahamrubilaralbarran/Desktop/DataSet Monopoly /Base_clientes_Monopoly.xlsx"
    
    # Verificar cuál de las rutas del archivo Excel existe para cargar el archivo
    if os.path.exists(ruta_excel_1):
        ruta_excel = ruta_excel_1
        print(f"Cargando archivo Excel desde {ruta_excel}")
    elif os.path.exists(ruta_excel_2):
        ruta_excel = ruta_excel_2
        print(f"Cargando archivo Excel desde {ruta_excel}")
    elif os.path.exists(ruta_excel_3):
        ruta_excel = ruta_excel_3
        print(f"Cargando archivo Excel desde {ruta_excel}")
    else:
        raise FileNotFoundError("Ninguna de las rutas de archivo Excel existe.")
    
    # Cargar el archivo Excel
    df = pd.read_excel(ruta_excel, engine="openpyxl")
    df.columns = df.iloc[0]  # Configurar la primera fila como encabezado
    df1 = df.iloc[1:].reset_index(drop=True)  # Eliminar la primera fila de encabezados redundantes
    
    # Asignar la ruta de salida predeterminada para el archivo Parquet
    ruta_parquet = ruta_parquet_1
    
    # Convertir y guardar en formato parquet
    df1.to_parquet(ruta_parquet, engine="pyarrow")
    print(f"Archivo convertido y guardado en {ruta_parquet}")

# Cargar el archivo Parquet (ya sea existente o recién creado) para verificar su contenido
df = pd.read_parquet(ruta_parquet, engine="pyarrow")
print(df.head(10))

### Como podemos ver en el head() anterior, tenemos la última columna repleta de valores NaN, por lo que vamos a borrar esa columna por completo.

In [ ]:
df.drop(df.columns[-1], axis=1, inplace=True)
df.head(5)

### Para poder proceder al análisis exploratorio del Dataframe, seleccionaremos las columnas más relevantes para encontrar respuesta a las preguntas expuestas en la primera fase.
### Para esto, crearemos un Array el cual va a contener todos los nombres de las columnas que se utilizarán para el análisis, se establece un nuevo Dataframe con las columnas seleccionadas.

In [ ]:
# Creamos el Array y se insertan el nombre de las columnas seleccionadas
columnas_permitidas = ['Id', 'Edad', 'Renta', 'Region', 'Sexo', 'TC', 'Cuentas', 'Hipotecario', 'Consumo', 'Debito', 
              'Ctacte', 'Antiguedad', 'Dualidad','FacCN_T01', 'FacCN_T02', 'FacCN_T03', 'FacCN_T04', 
              'FacCN_T05', 'FacCN_T06', 'FacCN_T07', 'FacCN_T08', 'FacCN_T09', 'FacCN_T10', 'FacCN_T11', 
              'FacCN_T12', 'FacCI_T01', 'FacCI_T02', 'FacCI_T03', 'FacCI_T04', 'FacCI_T05', 'FacCI_T06', 
              'FacCI_T07', 'FacCI_T08', 'FacCI_T09', 'FacCI_T10', 'FacCI_T11', 'FacCI_T12', 'TxsCN_T01', 
              'TxsCN_T02', 'TxsCN_T03', 'TxsCN_T04', 'TxsCN_T05', 'TxsCN_T06', 'TxsCN_T07', 'TxsCN_T08', 
              'TxsCN_T09', 'TxsCN_T10', 'TxsCN_T11', 'TxsCN_T12', 'TxsCI_T01', 'TxsCI_T02', 'TxsCI_T03', 
              'TxsCI_T04', 'TxsCI_T05', 'TxsCI_T06', 'TxsCI_T07', 'TxsCI_T08', 'TxsCI_T09', 'TxsCI_T10', 
              'TxsCI_T11', 'TxsCI_T12', 'UsoL1_T01','UsoL1_T02', 'UsoL1_T03', 'UsoL1_T04', 'UsoL1_T05', 
              'UsoL1_T06', 'UsoL1_T07', 'UsoL1_T08', 'UsoL1_T09', 'UsoL1_T10', 'UsoL1_T11', 'UsoL1_T12', 
              'UsoLI_T01', 'UsoLI_T02', 'UsoLI_T03', 'UsoLI_T04', 'UsoLI_T05', 'UsoLI_T06', 'UsoLI_T07', 
              'UsoLI_T08', 'UsoLI_T09', 'UsoLI_T10', 'UsoLI_T11', 'UsoLI_T12', 'CUPO_L1', 'CUPO_MX', 
              'PagoNac_T01', 'PagoNac_T02', 'PagoNac_T03', 'PagoNac_T04', 'PagoNac_T05', 'PagoNac_T06', 
              'PagoNac_T07', 'PagoNac_T08', 'PagoNac_T09', 'PagoNac_T10', 'PagoNac_T11', 'PagoNac_T12', 
              'PagoInt_T01', 'PagoInt_T02', 'PagoInt_T03', 'PagoInt_T04', 'PagoInt_T05', 'PagoInt_T06', 
              'PagoInt_T07', 'PagoInt_T08', 'PagoInt_T09', 'PagoInt_T10', 'PagoInt_T11', 'PagoInt_T12', 
              'FlgAct_T01', 'FlgAct_T02', 'FlgAct_T03', 'FlgAct_T04', 'FlgAct_T05', 'FlgAct_T06', 'FlgAct_T07', 
              'FlgAct_T08', 'FlgAct_T09', 'FlgAct_T10', 'FlgAct_T11', 'FlgAct_T12', 'FacAN_T01', 'FacAN_T02', 
              'FacAN_T03', 'FacAN_T04', 'FacAN_T05', 'FacAN_T06', 'FacAN_T07', 'FacAN_T08', 'FacAN_T09', 
              'FacAN_T10', 'FacAN_T11', 'FacAN_T12', 'FacAI_T01', 'FacAI_T02', 'FacAI_T03', 'FacAI_T04', 
              'FacAI_T05', 'FacAI_T06', 'FacAI_T07', 'FacAI_T08', 'FacAI_T09', 'FacAI_T10', 'FacAI_T11', 
              'FacAI_T12', 'target','IndRev_T12','IndRev_T11','IndRev_T10','IndRev_T09','IndRev_T08',
              'IndRev_T07','IndRev_T06','IndRev_T05','IndRev_T04','IndRev_T03','IndRev_T02','IndRev_T01']

# Se establece el nuevo nombre del Dataframe que contiene las columnas seleccionadas
df_acotado = df[columnas_permitidas]

# Iniciamos con el primer paso de la exploración de los datos
df_acotado.info()

### En la celda ejecutada anteriormente, se aprecian todos los tipos de datos del Dataframe. Ahora revisamos cuáles son los valores únicos dentro de estos datos.

In [ ]:
for i in df_acotado:
    print(f"{i} = {df_acotado[i].unique()} \n")

### En el resultado de la celda anterior, nos pudimos percatar que al mostrar los valores únicos, hay filas que no tienen valor asignado (NaN), por lo que ahora vamos a sumar todas las filas que tienen al menos una entrada de este tipo.

In [ ]:
df_acotado.isna().any(axis=1).sum()

# ⬆⬆⬆
### Esto nos indica que hay 19.862 filas que tienen al menos un valor en alguna de las columnas como NaN.

### La siguiente celda nos permite revisar cómo están compuestos los datos de cada columna, nos entrega información relevante como el promedio, valor mínimo, máximo, cuartiles y el tipo de dato. 

In [ ]:
for i in df_acotado:
  print(f"{i} = {df_acotado[i].describe()} \n")

### A continuación vamos a ramificar el Dataframe original para poder hacer un estudio con gráficos, ya que, se deben separar los valores numéricos y categoricos. Para esto vamos a hacer un Array para contener las columnas categóricas encontradas en el Dataframe.

In [ ]:
# Creación del Array
columnas_a_seleccionar = []

# Ciclo for para iterar sobre los nombres de las columnas que empiezan con "IndRev_"
for columna in df_acotado.columns:
    if columna.startswith('IndRev_'):
        columnas_a_seleccionar.append(columna)

# Seleccionar las columnas "IndRev_" en un nuevo Dataframe
df_categorico = df_acotado[columnas_a_seleccionar]

# Creamos un segundo Dataframe con las siguientes columnas
df_categorico2 = df_acotado[['Sexo', 'target']]

# Y concatenamos los Dataframe en unos solo
df_concatenado_categorico = pd.concat([df_categorico2, df_categorico], axis=1)
df_concatenado_categorico

### Sustraemos todas las columnas categóricas del Dataframe original para renombrarlo como un nuevo Dataframe de sólo columnas numéricas.

In [ ]:
df_acotado_numericas = df_acotado.drop(['Sexo','target','IndRev_T12','IndRev_T11','IndRev_T10','IndRev_T09','IndRev_T08','IndRev_T07','IndRev_T06','IndRev_T05','IndRev_T04','IndRev_T03','IndRev_T02','IndRev_T01'], axis=1)
df_acotado_numericas

### Revisamos el data frame de las variables numéricas.

In [ ]:
for i in df_acotado_numericas:
  print(f"{i} = {df_acotado_numericas[i].describe()} \n")

### Revisamos las correlaciones que tienen las variables numéricas en nuestro nuevo Dataframe.

In [ ]:
def plot_top_correlations(dataframe, top_n=30):
    # Calcular la matriz de correlación
    corr_matrix = dataframe.corr().abs()
    
    # Extraer las correlaciones superiores a la diagonal
    triangulo_matriz = corr_matrix.where(
        np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
    )
    
    # Encontrar las correlaciones más altas
    top_corr_pairs = (
        triangulo_matriz.unstack()
        .dropna()
        .sort_values(ascending=False)
        .head(top_n)
    )
    
    # Obtener las columnas de las correlaciones más altas
    top_columnas = list(set([index[0] for index in top_corr_pairs.index] + [index[1] for index in top_corr_pairs.index]))
    
    # Crear un nuevo dataframe con solo las columnas seleccionadas
    top_corr_dataframe = dataframe[top_columnas]
    
    # Calcular la matriz de correlación del nuevo dataframe
    top_corr_matriz = top_corr_dataframe.corr()
    
    # Dibujar el mapa de calor
    plt.figure(figsize=(15, 10))
    sb.heatmap(top_corr_matriz, annot=True, cmap='coolwarm', fmt='.2f', linewidths=.5)
    plt.title('Top {} Feature Correlations'.format(top_n))
    plt.show()

# Uso de la función
# Asegúrate de reemplazar 'your_dataframe' con el nombre de tu dataframe
plot_top_correlations(df_acotado_numericas)


In [ ]:
# Definimos las columnas que queremos analizar en detalle
columnas_para_filtrar = ['Edad', 'Region', 'Antiguedad']

# Iteramos sobre cada columna del DataFrame, excluyendo un gran conjunto de columnas
for columna in df_acotado.drop(df_acotado[['Id','FacCN_T01', 'FacCN_T02', 'FacCN_T03', 'FacCN_T04', 
              'FacCN_T05', 'FacCN_T06', 'FacCN_T07', 'FacCN_T08', 'FacCN_T09', 'FacCN_T10', 'FacCN_T11', 
              'FacCN_T12', 'FacCI_T01', 'FacCI_T02', 'FacCI_T03', 'FacCI_T04', 'FacCI_T05', 'FacCI_T06', 
              'FacCI_T07', 'FacCI_T08', 'FacCI_T09', 'FacCI_T10', 'FacCI_T11', 'FacCI_T12', 'TxsCN_T01', 
              'TxsCN_T02', 'TxsCN_T03', 'TxsCN_T04', 'TxsCN_T05', 'TxsCN_T06', 'TxsCN_T07', 'TxsCN_T08', 
              'TxsCN_T09', 'TxsCN_T10', 'TxsCN_T11', 'TxsCN_T12', 'TxsCI_T01', 'TxsCI_T02', 'TxsCI_T03', 
              'TxsCI_T04', 'TxsCI_T05', 'TxsCI_T06', 'TxsCI_T07', 'TxsCI_T08', 'TxsCI_T09', 'TxsCI_T10', 
              'TxsCI_T11', 'TxsCI_T12', 'UsoL1_T01','UsoL1_T02', 'UsoL1_T03', 'UsoL1_T04', 'UsoL1_T05', 
              'UsoL1_T06', 'UsoL1_T07', 'UsoL1_T08', 'UsoL1_T09', 'UsoL1_T10', 'UsoL1_T11', 'UsoL1_T12', 
              'UsoLI_T01', 'UsoLI_T02', 'UsoLI_T03', 'UsoLI_T04', 'UsoLI_T05', 'UsoLI_T06', 'UsoLI_T07', 
              'UsoLI_T08', 'UsoLI_T09', 'UsoLI_T10', 'UsoLI_T11', 'UsoLI_T12', 
              'PagoNac_T01', 'PagoNac_T02', 'PagoNac_T03', 'PagoNac_T04', 'PagoNac_T05', 'PagoNac_T06', 
              'PagoNac_T07', 'PagoNac_T08', 'PagoNac_T09', 'PagoNac_T10', 'PagoNac_T11', 'PagoNac_T12', 
              'PagoInt_T01', 'PagoInt_T02', 'PagoInt_T03', 'PagoInt_T04', 'PagoInt_T05', 'PagoInt_T06', 
              'PagoInt_T07', 'PagoInt_T08', 'PagoInt_T09', 'PagoInt_T10', 'PagoInt_T11', 'PagoInt_T12', 
              'FlgAct_T01', 'FlgAct_T02', 'FlgAct_T03', 'FlgAct_T04', 'FlgAct_T05', 'FlgAct_T06', 'FlgAct_T07', 
              'FlgAct_T08', 'FlgAct_T09', 'FlgAct_T10', 'FlgAct_T11', 'FlgAct_T12', 'FacAN_T01', 'FacAN_T02', 
              'FacAN_T03', 'FacAN_T04', 'FacAN_T05', 'FacAN_T06', 'FacAN_T07', 'FacAN_T08', 'FacAN_T09', 
              'FacAN_T10', 'FacAN_T11', 'FacAN_T12', 'FacAI_T01', 'FacAI_T02', 'FacAI_T03', 'FacAI_T04', 
              'FacAI_T05', 'FacAI_T06', 'FacAI_T07', 'FacAI_T08', 'FacAI_T09', 'FacAI_T10', 'FacAI_T11', 
              'FacAI_T12', 'IndRev_T12','IndRev_T11','IndRev_T10','IndRev_T09','IndRev_T08',
              'IndRev_T07','IndRev_T06','IndRev_T05','IndRev_T04','IndRev_T03','IndRev_T02','IndRev_T01']], axis=1):
    plt.figure(figsize=(30, 5))
    sb.histplot(df_acotado[columna], bins=200)
    plt.title(columna)
    plt.xlabel(columna)
    plt.ylabel('Frecuencia')

    # Si la columna está en la lista de columnas a filtrar, ajustamos los límites de los ejes x
    if columna in columnas_para_filtrar:
        valor_max = df_acotado[columna].max()
        valor_min = df_acotado[columna].min()
        plt.xticks(np.arange(valor_min, valor_max + 1, step=1))
        plt.show()

### Una vez visto los gráficos, ahora veremos la distribución de los datos, destacando los cuartiles, los valores extremos (outliers) y algunas estadísticas básicas que podremos ver de forma gráfica de tres características posiblemente importantes.

In [ ]:
# Lista de columnas a analizar
columnas_a_analizar = ['Edad', 'Region', 'Antiguedad']

# Definir subconjuntos de edad en más rangos
df_sub_edad_menor_20 = df_acotado[df_acotado['Edad'] < 20]
df_sub_edad_joven = df_acotado[(df_acotado['Edad'] >= 20) & (df_acotado['Edad'] < 30)]
df_sub_edad_adulto = df_acotado[(df_acotado['Edad'] >= 30) & (df_acotado['Edad'] <= 60)]
df_sub_edad_mayor = df_acotado[(df_acotado['Edad'] > 60) & (df_acotado['Edad'] <= 70)]
df_sub_edad_mayor_70 = df_acotado[df_acotado['Edad'] > 70]

# Transparencia para los scatterplots
alpha_value = 0.5

# Contar el número de regiones únicas, incluyendo NaN
num_regiones = df_acotado['Region'].nunique()

# Mapa de colores personalizado para las regiones, incluyendo NaN
colores_regiones = sb.color_palette("tab20", num_regiones + 1)  # Incluye un color para NaN

for columna in columnas_a_analizar:
    plt.figure(figsize=(20, 10))
    
    # Cálculo de la media y mediana
    media = df_acotado[columna].mean()
    mediana = df_acotado[columna].median()

    # Cálculo del rango intercuartílico (IQR)
    Q1 = df_acotado[columna].quantile(0.25)
    Q3 = df_acotado[columna].quantile(0.75)
    IQR = Q3 - Q1
    lim_inferior = Q1 - 1.5 * IQR
    lim_superior = Q3 + 1.5 * IQR
    
    # Boxplot con IQR destacado
    plt.subplot(2, 2, 1)
    sb.boxplot(x=df_acotado[columna], palette='Set2')

    # Resaltar el IQR en el boxplot
    plt.axvspan(Q1, Q3, color='lightgreen', alpha=0.3, label=f'IQR: {IQR:.2f} (Q1: {Q1:.2f}, Q3: {Q3:.2f})')
    
    # Líneas para los límites de los outliers
    plt.axvline(lim_inferior, color='red', linestyle='dashdot', label=f'Límite Inferior de Outliers: {lim_inferior:.2f}')
    plt.axvline(lim_superior, color='red', linestyle='dashdot', label=f'Límite Superior de Outliers: {lim_superior:.2f}')
    
    # Líneas para la media y mediana
    plt.axvline(media, color='blue', linestyle='-', label=f'Media: {media:.2f}', linewidth=2)
    plt.axvline(mediana, color='purple', linestyle='-', label=f'Mediana: {mediana:.2f}', linewidth=2)

    # Asegurarse de que los ticks en el eje X sean enteros (solo para la columna Región)
    if columna == 'Region':
        plt.xticks(np.arange(1, num_regiones + 1, step=1), [str(i) for i in range(1, num_regiones + 1)])
    
    plt.title(f'Boxplot de {columna}')
    plt.legend(loc='center left', bbox_to_anchor=(1.05, 0.5), borderaxespad=0)  # Leyenda fuera del gráfico
    
    # Scatterplot con transparencia y coloración por valores (Regiones ordenadas)
    plt.subplot(2, 2, 2)
    
    if columna == 'Region':
        # Ordenar las regiones y asignar colores
        df_acotado['Region'] = df_acotado['Region'].fillna(0)  # Llenar NaN con 0 temporalmente
        regiones_ordenadas = np.sort(df_acotado['Region'].unique())  # Ordenar las regiones numéricamente, NaN quedará en 0

        # Separar la etiqueta para Región NaN
        regiones_ordenadas = regiones_ordenadas[regiones_ordenadas != 0]  # Excluir temporalmente Región 0 (NaN)
        regiones_ordenadas = np.append(regiones_ordenadas, 0)  # Añadir Región 0 (NaN) al final

        for idx, region in enumerate(regiones_ordenadas):
            if region == 0:
                label = 'Región NaN'  # Etiqueta para NaN
            else:
                label = f'Región {int(region)}'  # Etiquetas para las regiones numéricas
            
            df_region = df_acotado[df_acotado['Region'] == region]
            plt.scatter(df_region['Id'], df_region['Region'], alpha=alpha_value, label=label, c=[colores_regiones[idx]])

        # Ajustar el eje Y para que muestre los números enteros del 1 al 13
        plt.yticks(np.arange(1, 14, step=1), [str(i) for i in range(1, 14)])
        plt.xticks(np.arange(0, df_acotado['Id'].max(), step=10000))  # Asegura que las regiones se muestren adecuadamente
    else:
        plt.scatter(df_acotado['Id'], df_acotado[columna], alpha=alpha_value, c=df_acotado[columna], cmap='viridis')

    # Resaltar IQR en el scatterplot
    plt.axhspan(Q1, Q3, color='lightgreen', alpha=0.3, label='Rango Intercuartílico (IQR)')
    plt.axhline(lim_inferior, color='red', linestyle='--', label='Límite Inferior de Outliers')
    plt.axhline(lim_superior, color='red', linestyle='--', label='Límite Superior de Outliers')

    # Líneas para la media y mediana en el scatterplot
    plt.axhline(media, color='blue', linestyle='-', label=f'Media: {media:.2f}', linewidth=2)
    plt.axhline(mediana, color='purple', linestyle='-', label=f'Mediana: {mediana:.2f}', linewidth=2)
    
    plt.title(f'Scatterplot de {columna}')
    plt.xlabel('Id')
    plt.ylabel(columna)
    plt.legend(loc='center left', bbox_to_anchor=(1.05, 0.5), borderaxespad=0)  # Leyenda fuera del gráfico

    # Añadir más ticks en el eje X para columnas específicas
    if columna == 'Edad':
        plt.xticks(np.arange(df_acotado['Edad'].min(), df_acotado['Edad'].max() + 1, step=5))
    elif columna == 'Region':
        plt.xticks(np.arange(1, num_regiones + 1, step=1))  # Mostrar regiones del 1 a num_regiones
    elif columna == 'Antiguedad':
        plt.xticks(np.arange(df_acotado['Antiguedad'].min(), df_acotado['Antiguedad'].max() + 1, step=2))

    # Scatterplot con subconjuntos por grupos de edad, añadiendo nuevos rangos
    plt.subplot(2, 2, 3)
    if columna == 'Edad':
        plt.scatter(df_sub_edad_menor_20['Id'], df_sub_edad_menor_20[columna], alpha=alpha_value, label='Edad < 20', c='lightblue')
        plt.scatter(df_sub_edad_joven['Id'], df_sub_edad_joven[columna], alpha=alpha_value, label='20 <= Edad < 30', c='blue')
        plt.scatter(df_sub_edad_adulto['Id'], df_sub_edad_adulto[columna], alpha=alpha_value, label='30 <= Edad <= 60', c='green')
        plt.scatter(df_sub_edad_mayor['Id'], df_sub_edad_mayor[columna], alpha=alpha_value, label='60 < Edad <= 70', c='orange')
        plt.scatter(df_sub_edad_mayor_70['Id'], df_sub_edad_mayor_70[columna], alpha=alpha_value, label='Edad > 70', c='red')
        plt.legend(loc='center left', bbox_to_anchor=(1.05, 0.5), borderaxespad=0)  # Leyenda fuera del gráfico
        plt.title(f'Scatterplot de {columna} por grupos de edad')

    elif columna == 'Region':
        # Scatterplot con valores NaN en negro
        for region in regiones_ordenadas:
            if region == 0:
                df_region_nan = df_acotado[df_acotado['Region'] == 0]  # Filtrar Región NaN
                plt.scatter(df_region_nan['Id'], df_region_nan['Region'], alpha=alpha_value, label='Región NaN', c='black')
            else:
                df_region = df_acotado[df_acotado['Region'] == region]
                plt.scatter(df_region['Id'], df_region[columna], alpha=alpha_value, label=f'Región {int(region)}')

        plt.legend(loc='center left', bbox_to_anchor=(1.05, 0.5), borderaxespad=0)  # Leyenda fuera del gráfico
        plt.title(f'Scatterplot de {columna} por Región')

    elif columna == 'Antiguedad':
        # Subconjuntos de antigüedad
        df_sub_antiguo = df_acotado[df_acotado['Antiguedad'] > 12]
        df_sub_nuevo = df_acotado[df_acotado['Antiguedad'] <= 12]
        plt.scatter(df_sub_antiguo['Id'], df_sub_antiguo[columna], alpha=alpha_value, label='Antigüedad > 12', c='orange')
        plt.scatter(df_sub_nuevo['Id'], df_sub_nuevo[columna], alpha=alpha_value, label='Antigüedad <= 12', c='purple')
        plt.legend(loc='center left', bbox_to_anchor=(1.05, 0.5), borderaxespad=0)  # Leyenda fuera del gráfico
        plt.title(f'Scatterplot de {columna} por Antigüedad')

    # Ajustar diseño para evitar solapamientos
    plt.tight_layout()  
    plt.show()

### A continuación revisaremos los datos de la columna Región para saber exactamente cómo se distribuyen los clientes a lo largo del país.

In [ ]:
region = df_acotado['Region']
region.unique()

### Revisamos un poco de estadística básica.

In [ ]:
# Eliminar los valores NaN en la columna 'Region'
df_acotado['Region'].dropna(inplace=True)

# Eliminar los valores 0 de la columna 'Region'
df_acotado = df_acotado[df_acotado['Region'] != 0]

# Mostrar las estadísticas descriptivas de la columna 'Region'
print(df_acotado['Region'].describe())

# Mostrar los valores únicos en la columna 'Region'
print(df_acotado['Region'].unique())


### Verificamos que estamos tratando el número de la región como un dato entero.

In [ ]:
region = df_acotado['Region']
region = region.astype(int)
region.unique()

### Y seleccionados los rangos que se mostrarán en el gráfico.

In [ ]:
# Los gráficos de torta contarán el número datos entre los rangos y lo mostraremos en un gráfico.
n1 = region.loc[region == 1].count()
n2 = region.loc[region == 2].count()
n3 = region.loc[region == 3].count()
n4 = region.loc[region == 4].count()
n5 = region.loc[region == 5].count()
n6 = region.loc[region == 6].count()
n7 = region.loc[region == 7].count()
n8 = region.loc[region == 8].count()
n9 = region.loc[region == 9].count()
n10 = region.loc[region == 10].count()
n11 = region.loc[region == 11].count()
n12 = region.loc[region == 12].count()
n13 = region.loc[region == 13].count()
print(n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13)

In [ ]:
#Formato del grafico circular
datos = [n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13]

# Separación de cada trozo de la torta al centro 
exp = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]

# Etiquetas de cada trozo
m = ["1","2","3","4","5","6","7","8","9","10","11","12","13"]

plt.figure(figsize=(12,12))

plt.title("Distribución del volumen de registros por región")
plt.pie(datos, labels = m, explode = exp, autopct='%2.1f%%')
plt.show()

# Fase 3 - Data Preparation

En la fase de Preparación de Datos consiste en transformar los datos crudos en un formato apto para análisis y modelado. Esto implica limpiar los datos (corrigiendo errores, completando valores faltantes), transformarlos (normalizando, estandarizando), construir muestras representativas y crear nuevas variables si es necesario.

Como pudimos apreciar anteriormente, todas las columnas poseen datos tipo object, por lo tanto, tenemos que transformar cada columna a su mejor tipo de dato. *IMPORTANTE* Por ahora vamos a trabajar solamente con algunas cuantas columnas que utilizaremos para hacer un tipo de testeo de imputación de datos, además de hacer un pequeño análisis antes de explorar todos los datos como conjunto.

In [ ]:
# Crear una copia del DataFrame para modificar los tipos de datos
df_nuevos_dtype = df_acotado.copy()

# Columnas que deben ser enteros
columnas_enteros = [
    'Id', 'Edad', 'Region', 'TC', 'Cuentas', 'Hipotecario', 
    'Consumo', 'Debito', 'Ctacte', 'Antiguedad', 'Dualidad', 'target'
]
for col in columnas_enteros:
    df_nuevos_dtype[col] = df_nuevos_dtype[col].astype('int64')

# Convertir columnas de tipo float64
columnas_flotantes = [col for col in df_nuevos_dtype.columns if col not in columnas_enteros + ['Sexo', 'IndRev_T12', 'IndRev_T11', 'IndRev_T10', 'IndRev_T09', 'IndRev_T08', 'IndRev_T07', 'IndRev_T06', 'IndRev_T05', 'IndRev_T04', 'IndRev_T03', 'IndRev_T02', 'IndRev_T01']]
df_nuevos_dtype[columnas_flotantes] = df_nuevos_dtype[columnas_flotantes].astype('float64')

# Convertir columnas de tipo object
columnas_objeto = ['Sexo', 'IndRev_T12', 'IndRev_T11', 'IndRev_T10', 'IndRev_T09', 'IndRev_T08', 'IndRev_T07', 'IndRev_T06', 'IndRev_T05', 'IndRev_T04', 'IndRev_T03', 'IndRev_T02', 'IndRev_T01']
df_nuevos_dtype[columnas_objeto] = df_nuevos_dtype[columnas_objeto].astype('object')

# Verificar tipos de datos
for i in df_nuevos_dtype:
    print(f"{i} = {df_nuevos_dtype[i].dtype} \n")


### [MODIFICAR] Ahora vamos a eliminar todas las filas que contengan valor nulo en la columna "Renta" y en la columna "Region", ya que según contexto, todos los clientes ingresados en el Dataset original, tienen alguna relación con la tenencia de tarjeta de crédito, la cual tiene como por prerrequisito, tener una renta mínima en el sistema.

### Tomaremos a la Región 13, ya que concentra la mayor cantidad de datos.

In [ ]:
#df_limpio = df_nuevos_dtype.dropna(subset=['Renta'])
#df_limpio = df_nuevos_dtype['Renta'].dropna(inplace=True)

df_acotado_x_region = df_nuevos_dtype[df_nuevos_dtype['Region'] == 13]

In [ ]:

for d in df_acotado_x_region:
    print(f"{d} = {df_acotado_x_region[d].info()} \n")
    print(f"{d} = {df_acotado_x_region[d].describe()} \n")

df_acotado_x_region.head(25)

### Vamos a dejar afuera algunas variables que no utilizaremos.

In [ ]:
#cantidad_nan_renta = df_acotado_x_region['Renta'].isnull().sum()
##cantidad_nan_region = df_acotado_x_region['Region'].isnull().sum()
#print("Cantidad de NaN en Renta:", cantidad_nan_renta)
#print("Cantidad de NaN en Region:", cantidad_nan_renta)

df_acotado_parte2 = df_acotado_x_region[['FacCN_T01', 'FacCN_T02', 'FacCN_T03', 'FacCN_T04', 'FacCN_T05', 
                                         'FacCN_T06', 'FacCN_T07', 'FacCN_T08', 'FacCN_T09', 'FacCN_T10', 
                                         'FacCN_T11', 'FacCN_T12', 'FacCI_T01', 'FacCI_T02', 'FacCI_T03', 
                                         'FacCI_T04', 'FacCI_T05', 'FacCI_T06', 'FacCI_T07', 'FacCI_T08', 
                                         'FacCI_T09', 'FacCI_T10', 'FacCI_T11', 'FacCI_T12']]

df_acotado_x_region = df_acotado_x_region.drop(df_acotado_x_region[['Dualidad', 'Consumo', 'Ctacte', 'Debito', 'Hipotecario',
'TxsCN_T01', 'TxsCN_T02', 'TxsCN_T03', 'TxsCN_T04', 'TxsCN_T05', 'TxsCN_T06', 'TxsCN_T07', 'TxsCN_T08', 'TxsCN_T09', 'TxsCN_T10', 'TxsCN_T11', 'TxsCN_T12',
'TxsCI_T01', 'TxsCI_T02', 'TxsCI_T03', 'TxsCI_T04', 'TxsCI_T05', 'TxsCI_T06', 'TxsCI_T07', 'TxsCI_T08', 'TxsCI_T09', 'TxsCI_T10', 'TxsCI_T11', 'TxsCI_T12',
'FacCN_T01', 'FacCN_T02', 'FacCN_T03', 'FacCN_T04', 'FacCN_T05', 'FacCN_T06', 'FacCN_T07', 'FacCN_T08', 'FacCN_T09', 'FacCN_T10','FacCN_T11', 'FacCN_T12', 
'FacCI_T01', 'FacCI_T02', 'FacCI_T03','FacCI_T04', 'FacCI_T05', 'FacCI_T06', 'FacCI_T07', 'FacCI_T08', 'FacCI_T09', 'FacCI_T10', 'FacCI_T11', 'FacCI_T12',
'UsoL1_T01','UsoL1_T02', 'UsoL1_T03', 'UsoL1_T04', 'UsoL1_T05', 'UsoL1_T06', 'UsoL1_T07', 'UsoL1_T08', 'UsoL1_T09', 'UsoL1_T10', 'UsoL1_T11', 'UsoL1_T12', 
'UsoLI_T01', 'UsoLI_T02', 'UsoLI_T03', 'UsoLI_T04', 'UsoLI_T05', 'UsoLI_T06', 'UsoLI_T07', 'UsoLI_T08', 'UsoLI_T09', 'UsoLI_T10', 'UsoLI_T11', 'UsoLI_T12', 
'PagoNac_T01', 'PagoNac_T02', 'PagoNac_T03', 'PagoNac_T04', 'PagoNac_T05', 'PagoNac_T06', 
'PagoNac_T07', 'PagoNac_T08', 'PagoNac_T09', 'PagoNac_T10', 'PagoNac_T11', 'PagoNac_T12', 
'PagoInt_T01', 'PagoInt_T02', 'PagoInt_T03', 'PagoInt_T04', 'PagoInt_T05', 'PagoInt_T06', 
'PagoInt_T07', 'PagoInt_T08', 'PagoInt_T09', 'PagoInt_T10', 'PagoInt_T11', 'PagoInt_T12', 
'FlgAct_T01', 'FlgAct_T02', 'FlgAct_T03', 'FlgAct_T04', 'FlgAct_T05', 'FlgAct_T06', 'FlgAct_T07', 
'FlgAct_T08', 'FlgAct_T09', 'FlgAct_T10', 'FlgAct_T11', 'FlgAct_T12', 'FacAN_T01', 'FacAN_T02', 
'FacAN_T03', 'FacAN_T04', 'FacAN_T05', 'FacAN_T06', 'FacAN_T07', 'FacAN_T08', 'FacAN_T09', 
'FacAN_T10', 'FacAN_T11', 'FacAN_T12', 'FacAI_T01', 'FacAI_T02', 'FacAI_T03', 'FacAI_T04', 
'FacAI_T05', 'FacAI_T06', 'FacAI_T07', 'FacAI_T08', 'FacAI_T09', 'FacAI_T10', 'FacAI_T11', 
'FacAI_T12', 'IndRev_T12','IndRev_T11','IndRev_T10','IndRev_T09','IndRev_T08',
'IndRev_T07','IndRev_T06','IndRev_T05','IndRev_T04','IndRev_T03','IndRev_T02','IndRev_T01']], axis=1)

# Concatenar df_acotado_x_region y df_acotado_parte2 horizontalmente
df_acotado_x_region = pd.concat([df_acotado_x_region, df_acotado_parte2], axis=1)

# Verificar los primeros registros para confirmar la unión
df_acotado_x_region.head(20)

In [ ]:
# Función para calcular los promedios mensuales
def calcular_promedios_mensuales(df):
    # Calcular el promedio de las columnas FacCN_T01 a FacCN_T12
    df['FacCN_mensual'] = df[['FacCN_T01', 'FacCN_T02', 'FacCN_T03', 'FacCN_T04', 'FacCN_T05', 
                              'FacCN_T06', 'FacCN_T07', 'FacCN_T08', 'FacCN_T09', 'FacCN_T10', 
                              'FacCN_T11', 'FacCN_T12']].mean(axis=1).round(0)
    
    # Calcular el promedio de las columnas FacCI_T01 a FacCI_T12
    df['FacCI_mensual'] = df[['FacCI_T01', 'FacCI_T02', 'FacCI_T03', 'FacCI_T04', 'FacCI_T05', 
                              'FacCI_T06', 'FacCI_T07', 'FacCI_T08', 'FacCI_T09', 'FacCI_T10', 
                              'FacCI_T11', 'FacCI_T12']].mean(axis=1).round(0)
    
    return df

# Función para calcular el gasto anual
def calcular_gasto_anual(df):
    # Sumar los valores de las columnas FacCN_T01 a FacCN_T12 para obtener FacCN_anual
    df['FacCN_anual'] = df[['FacCN_T01', 'FacCN_T02', 'FacCN_T03', 'FacCN_T04', 'FacCN_T05', 
                            'FacCN_T06', 'FacCN_T07', 'FacCN_T08', 'FacCN_T09', 'FacCN_T10', 
                            'FacCN_T11', 'FacCN_T12']].sum(axis=1).round(0)
    
    # Sumar los valores de las columnas FacCI_T01 a FacCI_T12 para obtener FacCI_anual
    df['FacCI_anual'] = df[['FacCI_T01', 'FacCI_T02', 'FacCI_T03', 'FacCI_T04', 'FacCI_T05', 
                            'FacCI_T06', 'FacCI_T07', 'FacCI_T08', 'FacCI_T09', 'FacCI_T10', 
                            'FacCI_T11', 'FacCI_T12']].sum(axis=1).round(0)
    
    return df

# Aplicar ambas funciones para obtener el DataFrame con los promedios mensuales y gastos anuales
df_acotado_x_region = calcular_promedios_mensuales(df_acotado_x_region)
df_acotado_x_region = calcular_gasto_anual(df_acotado_x_region)

# Eliminar las columnas FacCN_T01 a FacCN_T12 y FacCI_T01 a FacCI_T12
df_acotado_x_region = df_acotado_x_region.drop(df_acotado_x_region[['FacCN_T01', 'FacCN_T02', 'FacCN_T03', 'FacCN_T04', 'FacCN_T05', 
                                         'FacCN_T06', 'FacCN_T07', 'FacCN_T08', 'FacCN_T09', 'FacCN_T10', 
                                         'FacCN_T11', 'FacCN_T12', 'FacCI_T01', 'FacCI_T02', 'FacCI_T03', 
                                         'FacCI_T04', 'FacCI_T05', 'FacCI_T06', 'FacCI_T07', 'FacCI_T08', 
                                         'FacCI_T09', 'FacCI_T10', 'FacCI_T11', 'FacCI_T12']], axis=1)

# Mostrar las primeras 50 filas del DataFrame resultante
df_acotado_x_region.head(50)

### Se ha detectado una inconsistencia en la gestión de los cupos de tarjetas de crédito. La columna 'CUPO_MX', destinada a registrar el límite para compras internacionales, se encuentra expresada en dólares estadounidenses (USD). Sin embargo, la facturación de las compras internacionales (FacCI) se ha realizado en pesos chilenos (CLP), lo cual genera una discrepancia en la moneda utilizada para la administración de los límites y el registro de las transacciones.

### Revisamos la cantidad de 0s en la columna Region.

In [ ]:
# Contar los ceros en la columna Region
cantidad_ceros_region = (df_acotado_x_region['Region'] == 0).sum()

if cantidad_ceros_region > 0:
    print(f"Hay {cantidad_ceros_region} valores 0 en la columna Region.")
else:
    print("No hay valores 0 en la columna Region.")

### Hacemos una pequeña estadística de los datos por cada columna, para tener mejor idea de los datos con los que estamos trabajando.

In [ ]:
for i in df_acotado_x_region:
  print(f"{i} = {df_acotado_x_region[i].describe()} \n")

### Revisamos los valor únicos que tiene cada columna.

In [ ]:
for i in df_acotado_x_region:
    print(f"{i} = {df_acotado_x_region[i].unique()} \n")

In [ ]:
for i in df_acotado_x_region.columns:
    try:
        unique_values = df_acotado_x_region[i].unique()
        print(f"{i} = {unique_values} \n")
    except AttributeError as e:
        print(f"Error en la columna '{i}': {e}")

## Identificación de Valores Faltantes
### En este paso, identificamos las columnas que contienen valores faltantes dentro del dataset. Esto es importante ya que los algoritmos de Machine Learning generalmente no funcionan bien con datos incompletos. Visualizamos estos valores faltantes usando un mapa de calor para identificar qué columnas requieren atención.

In [ ]:
# Revisamos cuántos valores faltantes hay por columna
# Iterar por cada columna del DataFrame
for columna in df_acotado_x_region.columns:
    total_faltantes = df_acotado_x_region[columna].isna().sum()
    porcentaje_faltantes = (df_acotado_x_region[columna].isna().mean() * 100)
    tipo_dato = df_acotado_x_region[columna].dtype

    # Mostrar la información de la columna actual
    print(f"Columna: {columna}")
    print(f"  - Total Faltantes: {total_faltantes}")
    print(f"  - Porcentaje Faltantes: {porcentaje_faltantes:.2f}%")
    print(f"  - Tipo de Dato: {tipo_dato}")
    print("-" * 40)


### Eliminamos la columna Región.

In [ ]:
df_acotado_x_region = df_acotado_x_region.drop('Region', axis=1)
#df_acotado_x_region = df_acotado_x_region.drop('Id', axis=1)
# df_acotado_x_region.to_csv('/Users/herna/Desktop/df_acotado_new.csv')

## Detección y Manejo de Outliers
### Los outliers pueden influir negativamente en los resultados del análisis y el modelado. Aquí, utilizamos el método del rango intercuartil (IQR) para identificar y eliminar los outliers en las variables numéricas, asegurando que no distorsionen las relaciones entre las variables.

In [ ]:
# Suponiendo que df_acotado_x_region es el dataframe original
df_acotado_x_region_v2 = df_acotado_x_region.copy()

# Listado de columnas numéricas relevantes para detectar outliers
num_columnas = ['Edad', 'Renta', 'Antiguedad', 'CUPO_L1', 'CUPO_MX']

# Función para detectar outliers con el método IQR en cualquier columna numérica
def detectar_outliers(df, columna):
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    # Definir los límites para identificar los outliers
    outliers = df[(df[columna] < (Q1 - 1.5 * IQR)) | (df[columna] > (Q3 + 1.5 * IQR))]
    return outliers

# Visualización de outliers con gráficos de caja para una columna
def visualizar_outliers(df, columna):
    plt.figure(figsize=(10, 6))
    sb.boxplot(x=df[columna])
    plt.title(f'Boxplot de {columna}')
    plt.show()

# Detección, visualización y eliminación de outliers para todas las columnas numéricas
all_outliers = pd.DataFrame()  # Para acumular todos los outliers detectados

for columnas in num_columnas:
    # Detectar outliers en la columna actual
    outliers_col = detectar_outliers(df_acotado_x_region_v2, columnas)
    print(f"Outliers detectados en '{columnas}': {outliers_col.shape[0]}")
    
    # Visualizar los outliers en la columna actual
    visualizar_outliers(df_acotado_x_region_v2, columnas)
    
    # Acumular los outliers detectados
    all_outliers = pd.concat([all_outliers, outliers_col])

# Eliminar los outliers detectados de todas las columnas
df_acotado_x_region_v2 = df_acotado_x_region_v2[~df_acotado_x_region_v2.index.isin(all_outliers.index)]

# Mostrar el número de filas después de eliminar los outliers
print(f"Filas después de eliminar outliers: {df_acotado_x_region_v2.shape[0]}")

# Ver los primeros 25 registros después de eliminar outliers
df_acotado_x_region_v2.head(25)


### Imputamos con el modelo Regresión Lineal, porque es más confiable que la imputación con la media o mediana, y se basa en las relaciones entre múltiples variables del dataset, haciendo que las imputaciones sean más precisas.

In [ ]:
# Separar los datos con y sin valores faltantes en la columna 'Renta'
con_renta = df_acotado_x_region_v2[df_acotado_x_region_v2['Renta'].notnull()]
sin_renta = df_acotado_x_region_v2[df_acotado_x_region_v2['Renta'].isnull()]

# Definir las columnas predictoras (excluimos 'Renta', 'Unnamed: 0', 'target')
predictores = ['Edad', 'Sexo', 'Cuentas', 'Antiguedad']

# Separar las variables predictoras y la variable objetivo ('Renta')
X = con_renta[predictores]
y = con_renta['Renta']

# Preprocesamiento: OneHotEncoding para la columna 'Sexo' (es categórica)
preprocesador = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(drop='first'), ['Sexo'])
], remainder='passthrough')

# Crear un pipeline que combine preprocesamiento y modelo
pipeline = Pipeline(steps=[
    ('preprocessor', preprocesador),
    ('model', LinearRegression())
])

# Entrenar el modelo de regresión
pipeline.fit(X, y)

# Ahora usamos este modelo para predecir los valores faltantes de 'Renta'
X_faltantes = sin_renta[predictores]

# Predecir los valores de Renta faltantes
renta_pred = pipeline.predict(X_faltantes)

# Imputar los valores predichos en el dataframe original
df_acotado_x_region_v2.loc[df_acotado_x_region_v2['Renta'].isnull(), 'Renta'] = renta_pred

# Mostrar el dataframe con la columna 'Renta' imputada
df_acotado_x_region_v2.head(50)

### Vamos a borrar y verificar los nulos en las columnas del Array.

In [ ]:
columnas_a_checkear = ['Edad', 'Renta', 'Sexo', 'TC', 'Cuentas', 'Antiguedad', 'CUPO_L1', 'CUPO_MX', 'target']

# Mostrar la cantidad de filas antes de la limpieza
filas_iniciales = df_acotado_x_region_v2.shape[0]

# Eliminar las filas que contengan valores nulos en las columnas seleccionadas
df_acotado_x_region_v2 = df_acotado_x_region_v2.dropna(subset=columnas_a_checkear)

# Mostrar la cantidad de filas después de la limpieza
filas_limpias = df_acotado_x_region_v2.shape[0]

print(f"Filas iniciales: {filas_iniciales}")
print(f"Filas después de eliminar valores nulos en las columnas seleccionadas: {filas_limpias}")

porcentaje_reduccion = ((filas_iniciales / filas_limpias) * 100) - 100
porcentaje_reduccion_redondeado = round(porcentaje_reduccion, 1)

print(f"Porcentaje de reducción de datos: {porcentaje_reduccion_redondeado}%")

df_acotado_x_region_v2.head(50)

## Codificación de Variables Categóricas
### Para poder utilizar las variables categóricas en modelos de Machine Learning, es necesario transformarlas en representaciones numéricas. Para esto, utilizamos **Label Encoding** para variables categóricas con pocos valores (como 'Sexo')

In [ ]:
# Ver los valores únicos de la columna 'Region'
print(df_acotado_x_region_v2['Sexo'].unique())

# Ver la cantidad de valores únicos
print(f"Cantidad de valores únicos: {df_acotado_x_region_v2['Sexo'].nunique()}")

# Revisar si hay valores nulos en la columna 'Region'
print(f"Cantidad de valores nulos: {df_acotado_x_region_v2['Sexo'].isnull().sum()}")

# Hacer un conteo de la cantidad de veces que aparece cada valor único en la columna 'Region'
print(df_acotado_x_region_v2['Sexo'].value_counts())

In [ ]:
# Usamos Label Encoding para 'Sexo', ya que solo tiene unos pocos valores
label_encoder = LabelEncoder()
df_acotado_x_region_v2['Sexo'] = label_encoder.fit_transform(df_acotado_x_region_v2['Sexo'])
df_preparado = df_acotado_x_region_v2

df_preparado.head(20)

### Ahora salió una incógnita, ya que utilizamos el label encoding para las columnas, necesitamos primeramente identificar qué valores entre 0 y 1 serán los sexos "Masculino" y "Femenino"

In [ ]:
columnas_interes = ['Sexo']

conteo_ceros = df_preparado[columnas_interes].apply(lambda x: (x == True).sum())
conteo_unos = df_preparado[columnas_interes].apply(lambda x: (x == False).sum())
conteo_dos = df_preparado[columnas_interes].apply(lambda x: (x.isna()).sum())

resultados_conteo = pd.DataFrame({
    '0': conteo_ceros, # Femenino
    '1': conteo_unos, # Masculino
    '2': conteo_dos # EN el caso de haber quedado valores nulos.

})

print(resultados_conteo)
print("El valor con mayor cantidad serán los hombres ya que logramos apreciar anteriormente que hay más clientes de sexo masculino")
resultados_conteo.head(50)

In [ ]:
df_preparado.apply(lambda col: print(f"\nDescripción de la columna '{col.name}':\n", col.describe()))

In [ ]:
# Filtrar filas que no contengan valores negativos
df_filtrado = df_preparado[(df_preparado >= 0).all(axis=1)]


# Mostrar las primeras filas del dataframe filtrado
df_filtrado.head(50)

In [ ]:
df_filtrado.apply(lambda col: print(f"\nDescripción de la columna '{col.name}':\n", col.describe()))

In [ ]:
# Revisamos cuántos valores faltantes hay por columna
# Iterar por cada columna del DataFrame
for columna in df_filtrado.columns:
    total_faltantes = df_filtrado[columna].isna().sum()
    porcentaje_faltantes = (df_filtrado[columna].isna().mean() * 100)
    tipo_dato = df_filtrado[columna].dtype

    # Mostrar la información de la columna actual
    print(f"Columna: {columna}")
    print(f"  - Total Faltantes: {total_faltantes}")
    print(f"  - Porcentaje Faltantes: {porcentaje_faltantes:.2f}%")
    print(f"  - Tipo de Dato: {tipo_dato}")
    print("-" * 40)

df_filtrado.head(50)

### Revisaremos con gráficas el dataframe para guiarnos en los siguientes pasos.

In [ ]:
# Definimos las columnas que queremos analizar en detalle
columnas_para_filtrar = ['Edad', 'Region', 'Antiguedad']

# Iteramos sobre cada columna del DataFrame, excluyendo un gran conjunto de columnas
for columna in df_filtrado.drop(df_filtrado[['target']], axis=1):
    plt.figure(figsize=(30, 5))
    sb.histplot(df_filtrado[columna], bins=200)
    plt.title(columna)
    plt.xlabel(columna)
    plt.ylabel('Frecuencia')

    # Si la columna está en la lista de columnas a filtrar, ajustamos los límites de los ejes x
    if columna in columnas_para_filtrar:
        valor_max = df_filtrado[columna].max()
        valor_min = df_filtrado[columna].min()
        plt.xticks(np.arange(valor_min, valor_max + 1, step=1))
        plt.show()

## Seleccionaremos características, y convertiremos a enteros las que sean necesarias.

In [ ]:
# Columnas faltantes para conversión
conversion_col_faltantes = ['Renta', 'Sexo', 'CUPO_L1', 'CUPO_MX']
# Aplicar la transformación a cada columna
for columna in conversion_col_faltantes:
    # Redondear y convertir a int64
    df_filtrado[columna] = np.round(df_filtrado[columna]).astype('int64')

In [ ]:
df_filtrado.info()

In [ ]:
df_filtrado_v2 = df_filtrado.copy()

X = df_filtrado_v2.drop(columns=['Renta'])
y = df_filtrado_v2['Renta']

### Mostramos las características más relevantes para la predicción de la variable objetivo "y" en un conjunto de datos "X" utilizando una prueba estadística de regresión.

In [ ]:
mejores_cat = SelectKBest(score_func=f_regression, k=12)
fit = mejores_cat.fit(X, y)

puntaje_df = pd.DataFrame(fit.scores_)
columnas_df = pd.DataFrame(X.columns)

puntajes_cat = pd.concat([columnas_df, puntaje_df], axis=1)
puntajes_cat.columns = ['Descripción', 'Resultado']

puntajes_cat.nlargest(12, 'Resultado')

### Obtenemos importancia de características.

In [ ]:
modelo = ExtraTreesRegressor()
modelo.fit(X, y)

In [ ]:
print(modelo.feature_importances_)

Mostramos gráficamente.

In [ ]:
importancia_cat = pd.Series(modelo.feature_importances_, index=X.columns)
importancia_cat.nlargest(14).plot(kind='barh')
plt.show()

In [ ]:
#df_normalizado.head(20)
df_filtrado_v2.head(25)
for i in df_filtrado_v2:
    print(f"{i} = {df_filtrado_v2[i].unique()} \n")

Procederemos a tomar la variable contínua Edad, creando una nueva variable de tipo categórica ('Rango_Edad) con nuevas 'Features'.

In [ ]:
bins = [18, 30, 60, 100]
labels = ['Joven' , 'Adulto', 'Adulto Mayor']

df_filtrado_v2['Rango_Edad'] = pd.cut(df_filtrado_v2['Edad'], bins=bins, labels=labels)

orden_rangos = {'Joven': 0, 'Adulto': 1, 'Adulto Mayor': 2}

df_filtrado_v2['Rango_Edad'] = df_filtrado_v2['Rango_Edad'].map(orden_rangos)

df_filtrado_v2[['Edad', 'Rango_Edad']].head()

In [ ]:
def contar_rangos_edad(df_filtrado_v2 ):
    columna = 'Rango_Edad'
    if columna in df_filtrado_v2 .columns:
        conteos = df_filtrado_v2 [columna].value_counts()
        rangos_edad = [0, 1, 2]
        conteos_rangos = {rango: conteos.get(rango, 0) for rango in rangos_edad}
        return conteos_rangos
    else:
        raise ValueError(f"La columna '{columna}' no se encuentra en el DataFrame")

conteos_rangos_edad = contar_rangos_edad(df_filtrado_v2)
print(conteos_rangos_edad)

df_filtrado_v2.tail(50)

In [ ]:
df_filtrado_v2.drop(columns=['Unnamed: 0'], errors='ignore', inplace=True)
df_filtrado_v2

In [ ]:
df_filtrado_v2.info()

Pasamos la variable 'CUPO_MX', donde sus valores están reflejados en dólares estadounidenses, a pesos chilenos.

In [ ]:
valor_cambio = 816.36
cupo_mean = df_filtrado_v2['CUPO_MX'] * valor_cambio

df_filtrado_v2['CUPO_MX_CLP'] = cupo_mean.round(0).astype('int64')

df_filtrado_v2.head(100)

In [ ]:
df_filtrado_v2 = df_filtrado_v2.drop(columns=['target', 'Id'])
df_filtrado_v2

In [ ]:
df_filtrado_v2['FacCN_anual'] = df_filtrado_v2['FacCN_anual'].astype('int64')
df_filtrado_v2['FacCN_mensual'] = df_filtrado_v2['FacCN_mensual'].astype('int64')
df_filtrado_v2['FacCI_anual'] = df_filtrado_v2['FacCI_anual'].astype('int64')
df_filtrado_v2['FacCI_mensual'] = df_filtrado_v2['FacCI_mensual'].astype('int64')
df_filtrado_v2['Rango_Edad'] = df_filtrado_v2['Rango_Edad'].astype('int64')
df_filtrado_v2.info()

### Normalizaremos el dataset.

In [ ]:
# Copia del DataFrame original
df_normalizado = df_filtrado_v2.copy()

# Separar las columnas que van a usarse con StandardScaler
col_minmax = ['Edad', 'Antiguedad', 'CUPO_L1', 'CUPO_MX', 'Cuentas', 'Sexo', 'TC']

# Inicializar los escaladores
#scaler_standard = MinMaxScalerScaler()
scaler_minmax = MinMaxScaler()

# Aplicar StandardScaler
#df_normalizado[col_standard] = scaler_standard.fit_transform(df_normalizado[col_standard])

# Aplicar MinMaxScaler
df_normalizado[col_minmax] = scaler_minmax.fit_transform(df_normalizado[col_minmax])

# Modeling

# Evaluation

# Deployment